In [22]:
import datetime as dt
import os
import re
import timeit
from itertools import combinations


import xarray

import basd
import dask
import multiprocessing as mp
import pandas as pd
import rioxarray as rio
from rasterio.enums import Resampling
import numpy as np
import xarray as xr

In [14]:
data_path = '../../../OneDrive - PNNL/Documents/BASD/data'
obs_hist_path = 'pr_day_CESM2-WACCM_historical_r2i1p1f1_gn_20100101-20150101.nc'
sim_hist_path = 'pr_day_CESM2-WACCM_historical_r1i1p1f1_gn_20100101-20150101.nc'
sim_fut_path = 'pr_day_CESM2-WACCM_ssp370_r1i1p1f1_gn_20950101-21010101.nc'
pr_obs_hist = xr.open_dataset(os.path.join(data_path, obs_hist_path),
                              decode_coords = 'all')
pr_sim_hist = xr.open_dataset(os.path.join(data_path, sim_hist_path),
                              decode_coords = 'all')
pr_sim_fut = xr.open_dataset(os.path.join(data_path, sim_fut_path),
                             decode_coords = 'all')

C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\conventions.py:521: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
C:\Users\prim232\AppData\Local\Temp\ipykernel_20712\747905798.py:5: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  pr_obs_hist = xr.open_dataset(os.path.join(data_path, obs_hist_path),
C:\Users\prim232\AppData\Local\Temp\ipykernel_20712\747905798.py:7: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  pr_sim_hist = xr.open_dataset(os.path.join(data_path, sim_hist_path),
C:\Users\prim232\AppData\Local\Temp\ipykernel_20712\747905798.py:9: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  pr_sim_fut = xr.open_dataset(os.path.join(data_path, sim_fut_path),


In [19]:
# Making the data smaller for testing purposes
#pr_sim_fut = pr_sim_fut[dict(lat=range(75,115), lon=range(70,110))]
#pr_sim_hist = pr_sim_hist[dict(lat=range(75,115), lon=range(70,110))]
#pr_obs_hist = pr_obs_hist[dict(lat=range(75,115), lon=range(70,110))]
pr_sim_fut = pr_sim_fut[dict(lat=range(20), lon=range(20))]
pr_sim_hist = pr_sim_hist[dict(lat=range(20), lon=range(20))]
pr_obs_hist = pr_obs_hist[dict(lat=range(41), lon=range(45))]

In [29]:
pr_obs_hist

CRS.from_epsg(32735)

In [26]:
pr_sim_hist.rio.write_crs(4326, inplace=True)
pr_obs_hist.rio.write_crs(4326, inplace=True)
pr_sim_fut.rio.write_crs(4326, inplace=True)
sim_xy = pr_sim_hist.rename({'lon': 'x', 'lat': 'y'})
obs_xy = pr_obs_hist.rename({'lon': 'x', 'lat': 'y'})
sim_fut_xy = pr_sim_fut.rename({'lon': 'x', 'lat': 'y'})
obs_xy.rio.reproject_match(sim_fut_xy, resampling = Resampling.bilinear)
#pr_obs_hist.rio.crs

<xarray.Dataset>
Dimensions:      (x: 20, y: 20, time: 1826)
Coordinates:
  * x            (x) float64 0.0 1.25 2.5 3.75 5.0 ... 20.0 21.25 22.5 23.75
  * y            (y) float64 -90.0 -89.06 -88.12 -87.17 ... -73.98 -73.04 -72.09
  * time         (time) object 2010-01-01 00:00:00 ... 2015-01-01 00:00:00
    spatial_ref  int32 0
Data variables:
    pr           (time, y, x) float32 5.691e-07 5.691e-07 ... 8.221e-06
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  21900.0
    case_id:                5
    ...                     ...
    sub_experiment_id:      none
    table_id:               day
    tracking_id:            hdl:21.14100/8c71f502-bce3-4480-a06a-82f7188f6fe1
    variable_id:            pr
    variant_info:           CMIP6 CESM2 hindcast (1850-2015) with high-top at...
    variant_label:          r2i1p1f1

In [21]:
new_lat = np.linspace(pr_obs_hist.lat[0], pr_obs_hist.lat[-1], pr_sim_fut.sizes['lat'])
new_lon = np.linspace(pr_obs_hist.lon[0], pr_obs_hist.lon[-1], pr_sim_fut.sizes['lon'])
test = pr_obs_hist.interp(lat = new_lat, lon = new_lon, method = 'linear')
#test = pr_obs_hist.interp_like(pr_sim_fut)
test

<xarray.Dataset>
Dimensions:      (time: 1826, lat: 20, lon: 20, nbnd: 2)
Coordinates:
  * time         (time) object 2010-01-01 00:00:00 ... 2015-01-01 00:00:00
    time_bnds    (time, nbnd) object 2009-12-31 00:00:00 ... 2015-01-01 00:00:00
    lat_bnds     (lat, nbnd) float64 -90.0 -89.53 -88.49 ... -52.77 -51.83
    lon_bnds     (lon, nbnd) float64 -0.625 0.625 2.27 ... 52.73 54.38 55.62
    spatial_ref  int32 0
  * lat          (lat) float64 -90.0 -88.02 -86.03 ... -56.27 -54.29 -52.3
  * lon          (lon) float64 0.0 2.895 5.789 8.684 ... 46.32 49.21 52.11 55.0
Dimensions without coordinates: nbnd
Data variables:
    pr           (time, lat, lon) float64 5.691e-07 5.691e-07 ... 5.526e-05
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  21900.0
    case_id:                5
    ...                     ...
    sub_experiment_id:      none
    table_id:               day
    tracking_id:            hdl:21.14100/8c71f502-bce3-4480-a06a-82f7188f6fe1
    variable_id:            pr
    variant_info:           CMIP6 CESM2 hindcast (1850-2015) with high-top at...
    variant_label:          r2i1p1f1

In [5]:
params = basd.Parameters(lower_bound=0,
                         lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma')
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut, 'pr', params)

C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\coding\cftime_offsets.py:1130: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  return pd.date_range(
C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\coding\cftime_offsets.py:1130: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  return pd.date_range(
C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\coding\cftime_offsets.py:1130: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  return pd.date_range(


In [51]:
# Trying to find ideal chunk size
times = []
for i in range(1, 5):
    start = timeit.default_timer()
    chunk_result = ba.adjust_bias(lat_chunk_size=i, lon_chunk_size=i, n_jobs=-1)
    stop = timeit.default_timer()
    times.append(start-stop)
    print(f'Time for chunk size {i}: {stop-start}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  1

Time for chunk size 3: 261.4084363000002


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:  2.6min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed:  2.8min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:  3.0min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  18 out of  25 | elapsed:  3.7min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  20 out of  25 | elapsed:  3

Time for chunk size 4: 261.30030630000056


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  4.4min finished


In [52]:
times

[-242.51730260000022,
 -265.0072931000004,
 -261.4084363000002,
 -261.30030630000056]

In [10]:
chunk_result.to_netcdf(data_path + 'pr_sim_fut_ba.nc')

In [6]:
# Cells we're applying over
i_locations = np.ndindex(5, 5)

# Setting up list of dictionary items to pass to function
locs = []
for i_loc in i_locations:
    loc = dict(lat=i_loc[0], lon=i_loc[1])
    locs.append(loc)

# Vectorize bias adjust one location function
my_func = np.vectorize(ba.adjust_bias_one_location)

In [7]:
start = timeit.default_timer()
vector_out = my_func(locs)
stop = timeit.default_timer()
print(f'Vector Time: {stop-start}')

start = timeit.default_timer()
loop_out = []
for loc in locs:
    loop_out.append(ba.adjust_bias_one_location(loc))
stop = timeit.default_timer()
print(f'Loop Time: {stop-start}')

start = timeit.default_timer()
list_comp_out = [ba.adjust_bias_one_location(loc) for loc in locs]
stop = timeit.default_timer()
print(f'List Comp Time: {stop-start}')

start = timeit.default_timer()
map_out = list(map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Map Time: {stop-start}')

pool = mp.Pool()
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool Map Time: {stop-start}')

C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

Vector Time: 34.8868841


C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

Loop Time: 32.940850999999995


C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

List Comp Time: 32.36355610000001


C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

Map Time: 32.475623600000006
Pool Map Time: 25.726829699999996


In [53]:
pool = mp.Pool(1)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 1 Time: {stop-start}')

pool = mp.Pool(4)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 4 Time: {stop-start}')

pool = mp.Pool(8)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 8 Time: {stop-start}')

pool = mp.Pool(16)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 16 Time: {stop-start}')

Pool 1 Time: 39.50112799999988
Pool 4 Time: 16.773764999999912
Pool 8 Time: 18.433224600000358
Pool 16 Time: 24.62542999999914


In [8]:
pool = mp.Pool(8)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 8 Time: {stop-start}')